In [1]:
# try:
#     import piplite
#     await piplite.install(['ipywidgets'])
# except ImportError:
#     pass

In [2]:
%matplotlib inline

In [3]:
# import sys
# !{sys.executable} -m pip install elasticsearch plotly jupyter-dash ipywidgets
# import sys
# !{sys.executable} -m pip install ipywidgets elasticsearch numpy matplotlib pandas nltk scikit-learn
# !{sys.executable} -m pip install ipywidgets

# !{sys.executable} -m jupyter nbextension enable --py widgetsnbextension
# import sys
# !{sys.executable} -m jupyter nbextension enable --py --sys-prefix widgetsnbextension
# # ! pip install ipywidgets
# # ! jupyter nbextension enable --py --sys-prefix widgetsnbextension
# ! pip install scikit-learn
# ! pip install matplotlib
# ! pip install jupyter-dash
# ! pip install pandas
# ! pip install elasticsearch

In [4]:
import retrieve
import pandas as pd
from elasticsearch import Elasticsearch
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
import json

from collections import defaultdict

# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)
# simplefilter(action='ignore', category=SecurityWarning)

from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

from similarity import cosineSim, absoluteSim, manhattanSim, chi2Sim

In [5]:
def import_json(path):
    with open(path, 'r') as in_:
   		dataSetInfo = json.load(in_)
    return dataSetInfo

In [6]:
def connectES(password, host):
    es = Elasticsearch(
    host,
    ca_certs="/Users/tuomasketola/Dropbox/phd_files/searchEngineApp/relevance-structure-ranker/certs/http_ca.crt",
#     verify_certs=False,
    basic_auth=("elastic", password)
        )
    return es

In [7]:
def fetch_ranking_topk(query, index_name, model_name, k=100):
    ranking, weights = retrieve.retrieve(query, index_name, model_name)
#     print('retrieved {} documents form {}'.format(len(ranking), index_name))
    return ranking[:k], weights[:k]

In [8]:
def display_field_weights(weights):
    
    X_embedded = TSNE(n_components=2,  init='random', perplexity=3).fit_transform(weights)

    X_embedded_lst = X_embedded.tolist()
    xs = [x[0] for x in X_embedded_lst]
    ys = [x[1] for x in X_embedded_lst]

    return plt.scatter(xs, ys), xs, ys

In [9]:
def show_results(ranking, weights):
    to_display = []
    for document, doc_weights in zip(ranking, weights.tolist()):
        document_source = es.get(index=index_name, id=document[0])
        score = document[1]
        to_show = document_source['_source'][field_to_show]
        to_display.append((to_show, ': ',score, doc_weights, document[0]))
#         print(to_show, ': ',score, doc_weights)
    return to_display

In [10]:
ES_PASSWORD = '8kDCKZ2ZwFhRAQmFy6JP'
es = connectES(ES_PASSWORD, "https://localhost:9200")
if es.ping():
#     print('ES instance running succesfully')
    pass
else:
    print('ping did not work')

In [11]:
query = 'eurovision abba sweden'

In [12]:
index_name = 'imdb'

In [13]:
model_name = 'icfwG'

In [14]:
# field_to_show = 'title'
field_to_show = 'movie_name'

In [15]:
dataSetInfo = import_json('datasetInfo.json')[index_name]
fields = dataSetInfo['fields']

In [16]:
test =False
if test:
    ranking, weights = fetch_ranking_topk(query, index_name, model_name, k=20)
    weights = np.round(weights,2)
    to_display = show_results(ranking, weights)
    scatters, xs_scatter, ys_scatter = display_field_weights(weights)

In [17]:
from ipywidgets import IntSlider, HTML, Button, Layout, ButtonStyle,GridBox, Box, GridspecLayout, HBox, Text, Checkbox, VBox, Output
from IPython.display import display, clear_output

In [18]:
# potential_entities = ['lordi', 'Riva', 'carola', 'loreen', 'Lena']

In [19]:
header = HTML(
    value = "<b><font color='black'>{}</b>".format("Dubious Entity Search")
)

box = HBox([header], layout=Layout(height='auto', width='auto'))
# box.add_class("box_style")

In [20]:
search_bar = Text(
    value='wizard magic spells Daniel Radcliffe',
    placeholder='Type something',
    description='Query:',
    disabled=False
)

In [21]:
seed_entity_input = Text(
    value='Daniel Radcliffe',
    placeholder='Type something',
    description='Seed Entity:',
    disabled=False
)

In [22]:
dubious_entities_input = Text(
    value='Emma Watson,Robin Williams,Rupert Grint,Martin Freeman,Harry Potter,Hermione,Alladin,The boxer,Kelvin Moore,Bobbys Father,Gene ChardAllison Adams,Laura San Giacomo,Christophe Bon,Hackjin Mun',
    placeholder='Type something',
    description='Potential Dubious Entities',
    disabled=False
)

In [23]:
search_button = Button(
    description='Search',
    disabled=False,
    button_style='', 
    tooltip=''
)

In [24]:
more_info_button = Button(
    description='More Info',
    disabled=False,
    button_style='', 
    tooltip=''
)

In [25]:
rank_button = Button(
    description='Rank Enties',
    disabled=False,
    button_style='', 
    tooltip=''
)

In [26]:
aggregator_model_dropdown =widgets.Dropdown(
    options=['icfwG', 'icfwLA','linear'],
    value='icfwG',
    description='',
    disabled=False,
)

In [27]:
similarity_model_dropdown =widgets.Dropdown(
    options=['cosineSim', 'absoluteSim','manhattanSim','chi2Sim'],
    value='cosineSim',
    description='',
    disabled=False,
)

In [28]:
def findClusterLabels(xs, ys):
    clusterData = np.array(list(zip(xs,ys)))
    silScores = []
    for i in range(2,8):
        kmeans = KMeans(n_clusters=i, random_state=0).fit(clusterData)

        labels = kmeans.labels_

        silScore = silhouette_score(clusterData,labels)
        silScores.append((i, silScore, labels))
    silScoresSorted = sorted(silScores,key = lambda tup:tup[1], reverse=True)
    return silScoresSorted[0][2]

In [29]:
def bar_graph(b):
#     search_bar.value = 'ha'
    tooltip = b.tooltip
    global barOut
    with barOut:
        clear_output()
        document_id, weightsStr = tooltip.split('SPLIT')
        weights = [float(x) for x in weightsStr[1:-1].split(',')]
        fig, ax = plt.subplots(constrained_layout=True, figsize=(6, 4))
        bar = ax.bar(range(len(weights)),weights)
        if index_name == 'dbpedia':
            ax.set_xticks(range(0,5), [x.replace('_entities','') for x in fields])
        elif index_name == 'imdb':
            ax.set_xticks(range(0,len(fields)), [x.replace('movie_','') for x in fields])
        plt.show(fig)

In [35]:
def give_info(b):
    pass
    

In [30]:
def run_search(b):
    global seed_weights
    grid[18:28, 0:3].children = []
    query = search_bar.value
    seed_entity_ = seed_entity_input.value
    if seed_entity_ not in query:
        print('not ok')
        return False
    ranking, weights = fetch_ranking_topk(query, index_name, aggregator_model_dropdown.value, k=20)
    seed_weights = weights
    weights = np.round(weights,2)
    to_display = show_results(ranking, weights)
    result_titles = []
    result_checkboxes = []
    result_info_buttons = []
    
    
    # make scatter plot
    X_embedded = TSNE(n_components=2,  init='random', perplexity=3).fit_transform(weights)
    
    X_embedded_lst = X_embedded.tolist()
    xs = [x[0] for x in X_embedded_lst]
    ys = [x[1] for x in X_embedded_lst]
    
    labels = findClusterLabels(xs,ys)
    colors = ['b','g','r','c','m','y','k']
    color_dict = dict(zip(range(2,9),colors))
    colors = [color_dict[x+2] for x in labels.tolist()]   
    color_dict = {'b': 'blue', 'g': 'green', 'r': 'red', 'c': 'aquamarine', 'm': 'magenta', 'y': 'yellow', 'k': 'black'}
    
    global scatterOut
    with scatterOut:
        clear_output()
        fig, ax = plt.subplots(constrained_layout=True, figsize=(6, 4))
        for x,y,c in zip(xs,ys,colors):
            ax.scatter(x, y,color=c)
        plt.show(fig)
    result_count = 0
    for result in to_display:
        result_checkbox = Checkbox(
                    value=False,
                    description='Interesting',
                    disabled=False,
                    indent=False,
                    name=result[-1],
                    tooltip=result[-1] + 'SPLIT'+ str(result[-2]),
                    layout = widgets.Layout(width='80px')
                )
        result_title = HTML(
            value = result[0] if len(result[0]) < 30 else result[0][:30] + '...',
            )

        result_id = HTML(
            value = result[-1]
            )
        result_info_button = Button(
                description='More Info',
                disabled=False,
                button_style='', 
                tooltip=result[-1] + 'SPLIT'+ str(result[-2]),
                layout = widgets.Layout(width='80px')
            )
        result_info_button.style.button_color = color_dict[colors[result_count]]
        result_count += 1
        result_titles.append(VBox([result_title]))
        result_checkboxes.append(result_checkbox)
        result_info_buttons.append(result_info_button)
    grid[3:18, 0:3].children = result_titles
    grid[3:18, 3:4].children = result_checkboxes
    grid[3:18, 4:5].children = result_info_buttons
    

    _ = [result_info_button.on_click(bar_graph) for result_info_button in result_info_buttons]
    _ = [more_info_button.on_click(give_info) for result_info_button in result_info_buttons]
    _ = [result_checkbox.observe(checked) for result_checkbox in result_checkboxes]
    

In [31]:
def checked(b):
    if b['name'] == 'value' and b['new']:
        
        tooltip = b.owner.tooltip
        document_id, weightsStr = tooltip.split('SPLIT')
        query = search_bar.value
        to_show = '{query} - {id_} - {weights}'.format(id_=document_id, query=query, weights=weightsStr)
        grid[18:28, 0:3].children=tuple(list(grid[18:28, 0:3].children) + [HTML(value=to_show)])
        
    elif b['name'] == 'value' and not b['new']:
        tooltip = b.owner.tooltip
        document_id, weightsStr = tooltip.split('SPLIT')
        new_children = [x for x in list(grid[18:28, 0:3].children) if x.value.split(' - ')[1] != document_id]
        grid[18:28, 0:3].children = new_children

In [32]:
def rank_entities(b):
#     global potential_entities
    simModelDict = {
        "cosineSim": cosineSim,
        "absoluteSim" : absoluteSim,
        "manhattanSim" : manhattanSim,
        "chi2Sim" : chi2Sim
    }
    simModelName = 'absoluteSim'
    global entity_rel
    query = search_bar.value
    seed_entity = seed_entity_input.value
    potential_entities = dubious_entities_input.value.split(',')
    interesting_weights = [x.value.split(' - ')[2] for x in grid[18:28, 0:3].children]
    interesting_weights = [[float(x[1:-1]) for x in interesting_weight_str.split(',')] for interesting_weight_str in interesting_weights]
    interesting_weights = np.array(interesting_weights)
    rank_scores, entity_rel = simModelDict[similarity_model_dropdown.value].calc_similarity(fetch_ranking_topk, query, seed_entity, potential_entities, interesting_weights, index_name, model_name,es)
    rank_scores = sorted(rank_scores, key= lambda tup:tup[1], reverse=True)
    rank_scores_output = [HBox([HTML(value='{} - {}'.format(x[0], round(x[1],3))), more_info_button]) for x in rank_scores]
    grid[18:28, 5:10].children = rank_scores_output
    

In [33]:
result_titles = []
result_checkboxes = []
result_info_buttons = []
interesting_docs = []
    
scatterOut = Output()    
barOut = Output()


grid = GridspecLayout(35, 11, height='1500px', width='1000px')
grid[0:1,:] = box
grid[1:2,0:3] = seed_entity_input
grid[1:2,4:7] = search_bar
grid[1:2,8:10] = search_button
grid[2:3, 0:2] = dubious_entities_input
grid[2:3, 4:7] = aggregator_model_dropdown
grid[2:3, 8:10] = similarity_model_dropdown
grid[3:18, 0:3] = VBox(
    result_titles,
    layout=Layout(border='solid 1px black')
    )
grid[3:18, 3:4] = VBox(
    result_checkboxes,
    layout=Layout(border='solid 1px black')
    )
grid[3:18, 4:5] = VBox(
    result_info_buttons,
    layout=Layout(border='solid 1px black')
    )
                                   
grid[3:18,5:10] = VBox(
    [scatterOut, barOut],
    layout=Layout(border='solid 1px black')
)
grid[18:28, 0:3] = VBox(
    interesting_docs,
    layout=Layout(border='solid 1px black')
    )
grid[18:28, 5:10] = VBox(
    [],
    layout=Layout(border='solid 1px black')
)
grid[18:19, 3:4] = rank_button


query = search_bar.value
display(grid)
search_button.on_click(run_search)
rank_button.on_click(rank_entities)
# _ = [result_info_button.on_click(bar_graph) for result_info_button in grid[2:17, 4:5].children]



GridspecLayout(children=(HBox(children=(HTML(value="<b><font color='black'>Dubious Entity Search</b>"),), layo…

In [127]:
query_body = {
                'query': {
                      'multi_match':{
                        'query': "John AND McClane",
                        'fields': ['plot']}
                    }
                }

In [128]:
resp  = es.search(index='imdb', body = query_body, explain=True,size=10)

/var/folders/x5/wsnvl0vs5010p1lwp_8s1g1w0000gn/T/ipykernel_10839/2391497563.py:1: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  resp  = es.search(index='imdb', body = query_body, explain=True,size=10)


In [129]:
[x['_score'] for x in resp['hits']['hits']]

[19.605705,
 19.139019,
 19.023918,
 18.587767,
 10.748904,
 9.012202,
 6.366661,
 6.324498,
 6.304593,
 6.2754908]

In [130]:
query_body = {
    'query': {
          'query_string':{
            'query': 'John AND McClane',
            'fields': ['plot']}
        }
    }

In [131]:
resp  = es.search(index='imdb', body = query_body, explain=True,size=10)

/var/folders/x5/wsnvl0vs5010p1lwp_8s1g1w0000gn/T/ipykernel_10839/2391497563.py:1: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  resp  = es.search(index='imdb', body = query_body, explain=True,size=10)


In [132]:
[x['_score'] for x in resp['hits']['hits']]

[19.482428, 19.019955, 18.907429, 18.466743]

In [120]:
def make_full_query(q, ent):
    q_split = q.strip().split(' ')
    q = q.strip()
    if len(q_split) == 1:
        return q[0] + ' OR '+ ent
    else:
        return q.replace(' ',' OR ') + ' OR ' + '({})'.format(ent1.replace(' ', ' AND '))

In [121]:
query = 'magic wizard'
ent1 = 'Emma Watson'
ent2 = 'Hermione'

In [122]:
make_full_query(query, ent1)

'magic OR wizard OR (Emma AND Watson)'

In [123]:
make_full_query('magic wizard ', ent2)

'magic OR wizard OR (Emma AND Watson)'